In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from delta import *

In [2]:
builder = SparkSession.builder \
    .appName("Checkins to Silver") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:3.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
def load_checkin_data(spark, path):
    schema = StructType([
        StructField("business_id", StringType(), True),
        StructField("date", StringType(), True)
    ])
    return spark.read.json(path, schema=schema)

df = load_checkin_data(spark, "D:/Project/delta_lake/bronze/yelp_academic_dataset_checkin.json")
df.show(5)

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:...|
|--0iUa4sNDFiZFrAd...|2010-09-13 21:43:...|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:...|
|--7PUidqRWpRSpXeb...|2011-02-15 17:12:...|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:...|
+--------------------+--------------------+
only showing top 5 rows



In [4]:
def transform_dates(df):
    # Split the date string into individual timestamps
    df = df.withColumn("date_array", split(col("date"), ", "))
    
    # Explode the array to create individual rows for each checkin
    df = df.withColumn("checkin_timestamp", explode(col("date_array")))
    
    # Convert to timestamp and extract components
    df = df.withColumn("checkin_timestamp", to_timestamp(col("checkin_timestamp"))) \
        .withColumn("year", year(col("checkin_timestamp"))) \
        .withColumn("month", month(col("checkin_timestamp"))) \
        .withColumn("day", dayofmonth(col("checkin_timestamp"))) \
        .withColumn("hour", hour(col("checkin_timestamp"))) \
        .withColumn("day_of_week", date_format(col("checkin_timestamp"), "EEEE"))
    
    return df.drop("date_array", "date")

df = transform_dates(df)
df.show(5)


+--------------------+-------------------+----+-----+---+----+-----------+
|         business_id|  checkin_timestamp|year|month|day|hour|day_of_week|
+--------------------+-------------------+----+-----+---+----+-----------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:56|2020|    3| 13|  21|     Friday|
|---kPU91CF4Lq2-Wl...|2020-06-02 22:18:06|2020|    6|  2|  22|    Tuesday|
|---kPU91CF4Lq2-Wl...|2020-07-24 22:42:27|2020|    7| 24|  22|     Friday|
|---kPU91CF4Lq2-Wl...|2020-10-24 21:36:13|2020|   10| 24|  21|   Saturday|
|---kPU91CF4Lq2-Wl...|2020-12-09 21:23:33|2020|   12|  9|  21|  Wednesday|
+--------------------+-------------------+----+-----+---+----+-----------+
only showing top 5 rows



In [5]:
def feature_engineering(df):
    # Time period categorization
    df = df.withColumn("time_period",
        when((col("hour") >= 6) & (col("hour") < 12), "Morning")
        .when((col("hour") >= 12) & (col("hour") < 17), "Afternoon")
        .when((col("hour") >= 17) & (col("hour") < 22), "Evening")
        .otherwise("Night"))
    
    # Weekend flag
    df = df.withColumn("is_weekend",
        when(col("day_of_week").isin(["Saturday", "Sunday"]), True)
        .otherwise(False)) 
    
    # Business checkin frequency
    window_daily = Window.partitionBy("business_id", "year", "month", "day")
    df = df.withColumn("daily_checkins", count("*").over(window_daily))
    
    return df

df = feature_engineering(df)
df.show(5)


+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------+--------------+
|         business_id|  checkin_timestamp|year|month|day|hour|day_of_week|time_period|is_weekend|daily_checkins|
+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------+--------------+
|---kPU91CF4Lq2-Wl...|2020-12-09 21:23:33|2020|   12|  9|  21|  Wednesday|    Evening|     false|             1|
|--0iUa4sNDFiZFrAd...|2013-11-18 06:34:08|2013|   11| 18|   6|     Monday|    Morning|     false|             1|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:17|2013|    6| 14|  23|     Friday|      Night|     false|             1|
|--7PUidqRWpRSpXeb...|2014-05-02 15:49:55|2014|    5|  2|  15|     Friday|  Afternoon|     false|             1|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:11|2014|    4| 21|  20|     Monday|    Evening|     false|             1|
+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------

In [6]:
def calculate_metrics(df):
    # Daily aggregations
    daily_metrics = df.groupBy("business_id", "year", "month", "day") \
        .agg(
            count("*").alias("total_checkins"),
            countDistinct("hour").alias("unique_hours"),
            sum(when(col("is_weekend"), 1).otherwise(0)).alias("weekend_checkins"),
            sum(when(col("time_period") == "Morning", 1).otherwise(0)).alias("morning_checkins"),
            sum(when(col("time_period") == "Afternoon", 1).otherwise(0)).alias("afternoon_checkins"),
            sum(when(col("time_period") == "Evening", 1).otherwise(0)).alias("evening_checkins"),
            sum(when(col("time_period") == "Night", 1).otherwise(0)).alias("night_checkins")
        )
    
    return daily_metrics

df_metrics = calculate_metrics(df)
df_metrics.show(5)


+--------------------+----+-----+---+--------------+------------+----------------+----------------+------------------+----------------+--------------+
|         business_id|year|month|day|total_checkins|unique_hours|weekend_checkins|morning_checkins|afternoon_checkins|evening_checkins|night_checkins|
+--------------------+----+-----+---+--------------+------------+----------------+----------------+------------------+----------------+--------------+
|-8oIIpYJAXWg-G3gl...|2021|    1| 22|             1|           1|               0|               0|                 0|               0|             1|
|-HUDQ5eek6Edz3zuN...|2013|    5|  9|             1|           1|               0|               0|                 0|               1|             0|
|-OKB11ypR4C8wWlon...|2017|   10| 21|             2|           2|               2|               1|                 0|               0|             1|
|-Or44IdY51Ukd618k...|2020|    1| 12|             1|           1|               1|            

In [7]:
def validate_data(df):
    df = df.filter(
        (length(col("business_id")) > 0) &
        (col("checkin_timestamp").isNotNull()) &
        (col("year") >= 2004) &  # Yelp founding year
        (col("year") <= year(current_date())) &
        (col("hour").between(0, 23))
    )
    return df

df = validate_data(df)
df.show(5)


+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------+--------------+
|         business_id|  checkin_timestamp|year|month|day|hour|day_of_week|time_period|is_weekend|daily_checkins|
+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------+--------------+
|---kPU91CF4Lq2-Wl...|2020-12-09 21:23:33|2020|   12|  9|  21|  Wednesday|    Evening|     false|             1|
|--0iUa4sNDFiZFrAd...|2013-11-18 06:34:08|2013|   11| 18|   6|     Monday|    Morning|     false|             1|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:17|2013|    6| 14|  23|     Friday|      Night|     false|             1|
|--7PUidqRWpRSpXeb...|2014-05-02 15:49:55|2014|    5|  2|  15|     Friday|  Afternoon|     false|             1|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:11|2014|    4| 21|  20|     Monday|    Evening|     false|             1|
+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------

In [8]:
def quality_checks(df):
    null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) 
                           for c in df.columns])
    
    time_dist = df.groupBy("time_period") \
        .agg(count("*").alias("checkin_count")) \
        .orderBy("time_period")
    
    weekday_dist = df.groupBy("day_of_week") \
        .agg(count("*").alias("checkin_count")) \
        .orderBy("day_of_week")
    
    print("Null Counts:")
    null_counts.show()
    print("\nTime Period Distribution:")
    time_dist.show()
    print("\nWeekday Distribution:")
    weekday_dist.show()
    
    return df

df = quality_checks(df)


Null Counts:
+-----------+-----------------+----+-----+---+----+-----------+-----------+----------+--------------+
|business_id|checkin_timestamp|year|month|day|hour|day_of_week|time_period|is_weekend|daily_checkins|
+-----------+-----------------+----+-----+---+----+-----------+-----------+----------+--------------+
|          0|                0|   0|    0|  0|   0|          0|          0|         0|             0|
+-----------+-----------------+----+-----+---+----+-----------+-----------+----------+--------------+


Time Period Distribution:
+-----------+-------------+
|time_period|checkin_count|
+-----------+-------------+
|  Afternoon|      2366772|
|    Evening|      4705429|
|    Morning|       389729|
|      Night|      5894945|
+-----------+-------------+


Weekday Distribution:
+-----------+-------------+
|day_of_week|checkin_count|
+-----------+-------------+
|     Friday|      1959015|
|     Monday|      1491993|
|   Saturday|      2810469|
|     Sunday|      2480701|
|   T

In [9]:
df.show(20)

+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------+--------------+
|         business_id|  checkin_timestamp|year|month|day|hour|day_of_week|time_period|is_weekend|daily_checkins|
+--------------------+-------------------+----+-----+---+----+-----------+-----------+----------+--------------+
|---kPU91CF4Lq2-Wl...|2020-12-09 21:23:33|2020|   12|  9|  21|  Wednesday|    Evening|     false|             1|
|--0iUa4sNDFiZFrAd...|2013-11-18 06:34:08|2013|   11| 18|   6|     Monday|    Morning|     false|             1|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:17|2013|    6| 14|  23|     Friday|      Night|     false|             1|
|--7PUidqRWpRSpXeb...|2014-05-02 15:49:55|2014|    5|  2|  15|     Friday|  Afternoon|     false|             1|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:11|2014|    4| 21|  20|     Monday|    Evening|     false|             1|
|--7jw19RH9JKXgFoh...|2014-09-30 14:41:47|2014|    9| 30|  14|    Tuesday|  Afternoon|     false

In [10]:
df_metrics.show(20)

+--------------------+----+-----+---+--------------+------------+----------------+----------------+------------------+----------------+--------------+
|         business_id|year|month|day|total_checkins|unique_hours|weekend_checkins|morning_checkins|afternoon_checkins|evening_checkins|night_checkins|
+--------------------+----+-----+---+--------------+------------+----------------+----------------+------------------+----------------+--------------+
|-8oIIpYJAXWg-G3gl...|2021|    1| 22|             1|           1|               0|               0|                 0|               0|             1|
|-HUDQ5eek6Edz3zuN...|2013|    5|  9|             1|           1|               0|               0|                 0|               1|             0|
|-OKB11ypR4C8wWlon...|2017|   10| 21|             2|           2|               2|               1|                 0|               0|             1|
|-Or44IdY51Ukd618k...|2020|    1| 12|             1|           1|               1|            

In [11]:
# Save detailed checkins
df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("D:/Project/delta_lake/silver/checkins_detailed")

# Save daily metrics
df_metrics.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("D:/Project/delta_lake/silver/checkins_metrics")
